In [ ]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

In [ ]:
!unzip datasource.zip -d dealership_data 

In [ ]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [ ]:
# Add the CSV extract function below
def extract_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [ ]:
# Add the JSON extract function below
def extract_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

In [ ]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.
def extract_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

In [ ]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [ ]:
# Add the transform function below
def transform(data):
    data['price'] = round(data.price,2)
    return data

In [ ]:
# Add the load function below
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

In [ ]:
# Add the log function below
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [ ]:

log('ETL process started')

# Log that you have started the Extract step
log('extract started')
# Call the Extract function
extracted_data=extract()
# Log that you have completed the Extract step
extracted_data
log("extract completed")

# Log that you have started the Transform step
log("transformation started")
# Call the Transform function
transformed_data=transform(extracted_data)
# Log that you have completed the Transform step
log("transformation completed")


# Log that you have started the Load step
log("loading started")

# Call the Load function
load(targetfile,transformed_data)
# Log that you have completed the Load step
log("load completed")


# Log that you have completed the ETL process
log("ETL process completed")
